# Basic Classification Example with TensorFlow

This notebook is a companion of [A Visual and Interactive Guide to the Basics of Neural Networks](https://jalammar.github.io/visual-interactive-guide-basics-neural-networks/).

This is an example of how to do classification on a simple dataset in TensorFlow. Basically, we're building a model to help a friend choose a house to buy. She has given us the table below of houses and whether she likes them or not. We're to build a model that takes a house area and number of bathrooms as input, and outputs a prediction of whether she would like the house or not.

| Area (sq ft) (x1) | Bathrooms (x2) | Label (y) |
 | --- | --- | --- |
 | 2,104 |  3 | Good |
 | 1,600 |  3 | Good |
 | 2,400 |  3 | Good |
 | 1,416 | 	2 | Bad |
 | 3,000 | 	4 | Bad |
 | 1,985 | 	4 | Good |
 | 1,534 | 	3 | Bad |
 | 1,427 | 	3 | Good |
 | 1,380 | 	3 | Good |
 | 1,494 | 	3 | Good |
 
 
 
 We'll start by loading our favorite libraries

In [109]:
%matplotlib inline               
import pandas as pd              # A beautiful library to help us work with data as tables
import numpy as np               # So we can use number matrices. Both pandas and TensorFlow need it. 
import matplotlib.pyplot as plt  # Visualize the things
import tensorflow as tf          # Fire from the gods


We'll then load the house data CSV. Pandas is an incredible library that gives us great flexibility in dealing with table-like data. We load tables (or csv files, or excel sheets) into a "data frame", and process it however we like. You can think of it as a programatic way to do a lot of the things you previously did with Excel.

In [110]:
dataframe = pd.read_csv("input.csv") # Let's have Pandas load our dataset as a dataframe
dataframe = dataframe.drop(["Date", "Floors"], axis=1) # Remove columns we don't care about
dataframe = dataframe.replace({',': ''}, regex=True)
dataframe = dataframe.apply(pd.to_numeric)
dataframe # Let's have the notebook show us how the dataframe looks now

,Calories Burned,Steps,Distance,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories
0,2615,5527,4.41,739,153,42,31,1134
1,2372,5175,3.61,806,118,21,27,816
2,3045,9914,7.52,577,238,43,41,1659
3,2245,2245,1.56,897,174,0,0,709
4,2940,6629,5.23,622,171,77,46,1532
5,3114,7061,4.92,528,110,116,68,1710
6,2642,6293,4.39,598,183,25,31,1132
7,2229,1941,1.35,697,132,21,5,670
8,2481,4583,3.77,803,86,44,38,939
9,2313,1804,1.26,762,119,45,14,856


The dataframe now only has the features. Let's introduce the labels.

In [136]:
target_sleep = 8 * 60 # Sleep for 8 hours
labels = pd.read_csv("output.csv") # Let's have Pandas load our dataset as a dataframe
labels = labels.drop(["Date", "Minutes Awake", "Number of Awakenings", "Time in Bed"], axis=1) # Remove columns we don't care about
labels["WELL"] = (labels['Minutes Asleep'] >= target_sleep).astype(int)
labels.loc[:, ("BAD")] = labels["WELL"] == 0           # y2 is the negation of y1
labels.loc[:, ("BAD")] = labels["BAD"].astype(int)    # Turn TRUE/FALSE values into 1/0
labels = labels.drop(["Minutes Asleep"], axis=1) # Remove columns we don't care about
labels # Let's have the notebook show us how the dataframe looks now

,WELL,BAD
0,0,1
1,1,0
2,1,0
3,0,1
4,0,1
5,1,0
6,1,0
7,1,0
8,1,0
9,1,0


Now that we have all our data in the dataframe, we'll need to shape it in matrices to feed it to TensorFlow

In [137]:
inputX = dataframe.as_matrix()
inputY = labels.as_matrix()

So now our input matrix looks like this:

In [138]:
inputX

array([[  2.61500000e+03,   5.52700000e+03,   4.41000000e+00,
          7.39000000e+02,   1.53000000e+02,   4.20000000e+01,
          3.10000000e+01,   1.13400000e+03],
       [  2.37200000e+03,   5.17500000e+03,   3.61000000e+00,
          8.06000000e+02,   1.18000000e+02,   2.10000000e+01,
          2.70000000e+01,   8.16000000e+02],
       [  3.04500000e+03,   9.91400000e+03,   7.52000000e+00,
          5.77000000e+02,   2.38000000e+02,   4.30000000e+01,
          4.10000000e+01,   1.65900000e+03],
       [  2.24500000e+03,   2.24500000e+03,   1.56000000e+00,
          8.97000000e+02,   1.74000000e+02,   0.00000000e+00,
          0.00000000e+00,   7.09000000e+02],
       [  2.94000000e+03,   6.62900000e+03,   5.23000000e+00,
          6.22000000e+02,   1.71000000e+02,   7.70000000e+01,
          4.60000000e+01,   1.53200000e+03],
       [  3.11400000e+03,   7.06100000e+03,   4.92000000e+00,
          5.28000000e+02,   1.10000000e+02,   1.16000000e+02,
          6.80000000e+01,   1.7

And our labels matrix looks like this:

In [139]:
# inputY = inputY.reshape(-1, 1)
inputY

array([[0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1]])

Let's prepare some parameters for the training process

In [146]:
# Parameters
learning_rate = 0.00001
training_epochs = 200000
display_step = 5000
n_samples = inputY.size

And now to define the TensorFlow operations. Notice that this is a declaration step where we tell TensorFlow how the prediction is calculated. If we execute it, no calculation would be made. It would just acknowledge that it now knows how to do the operation.

In [147]:
x = tf.placeholder(tf.float32, [None, 8])   # Okay TensorFlow, we'll feed you an array of examples. Each example will
                                            # be an array of two float values (area, and number of bathrooms).
                                            # "None" means we can feed you any number of examples
                                            # Notice we haven't fed it the values yet
            
W = tf.Variable(tf.zeros([8, 2]))           # Maintain a 2 x 2 float matrix for the weights that we'll keep updating 
                                            # through the training process (make them all zero to begin with)
    
b = tf.Variable(tf.zeros([2]))              # Also maintain two bias values

y_values = tf.add(tf.matmul(x, W), b)       # The first step in calculating the prediction would be to multiply
                                            # the inputs matrix by the weights matrix then add the biases
    
y = tf.nn.softmax(y_values)                 # Then we use softmax as an "activation function" that translates the
                                            # numbers outputted by the previous layer into probability form
    
y_ = tf.placeholder(tf.float32, [None,2])   # For training purposes, we'll also feed you a matrix of labels

Let's specify our cost function and use Gradient Descent

In [148]:

# Cost function: Mean squared error
cost = tf.reduce_sum(tf.pow(y_ - y, 2))/(2*n_samples)
# Gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)


In [149]:
# Initialize variabls and tensorflow session
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


*Drum roll*

And now for the actual training

In [150]:
for i in range(training_epochs):  
    sess.run(optimizer, feed_dict={x: inputX, y_: inputY}) # Take a gradient descent step using our inputs and labels

    # That's all! The rest of the cell just outputs debug messages. 
    # Display logs per epoch step
    if (i) % display_step == 0:
        cc = sess.run(cost, feed_dict={x: inputX, y_:inputY})
        print "Training step:", '%04d' % (i), "cost=", "{:.9f}".format(cc) #, \"W=", sess.run(W), "b=", sess.run(b)

print "Optimization Finished!"
training_cost = sess.run(cost, feed_dict={x: inputX, y_: inputY})
print "Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n'


Training step: 0000 cost= 0.150000006
Training step: 5000 cost= 0.150000006
Training step: 10000 cost= 0.150000006
Training step: 15000 cost= 0.150000006
Training step: 20000 cost= 0.150000006
Training step: 25000 cost= 0.150000006
Training step: 30000 cost= 0.150000006
Training step: 35000 cost= 0.150000006
Training step: 40000 cost= 0.150000006
Training step: 45000 cost= 0.150000006
Training step: 50000 cost= 0.150000006
Training step: 55000 cost= 0.150000006
Training step: 60000 cost= 0.150000006
Training step: 65000 cost= 0.150000006
Training step: 70000 cost= 0.150000006
Training step: 75000 cost= 0.150000006
Training step: 80000 cost= 0.150000006
Training step: 85000 cost= 0.150000006
Training step: 90000 cost= 0.150000006
Training step: 95000 cost= 0.150000006
Training step: 100000 cost= 0.150000006
Training step: 105000 cost= 0.150000006
Training step: 110000 cost= 0.150000006
Training step: 115000 cost= 0.150000006
Training step: 120000 cost= 0.150000006
Training step: 125000 

Now the training is done. TensorFlow is now holding on to our trained model (Which is basically just the defined operations, plus the variables W and b that resulted from the training process).

Is a cost value of 0.109537 good or bad? I have no idea. At least it's better than the first cost value of 0.114958666. Let's use the model on our dataset to see how it does, though:

In [151]:
sess.run(y, feed_dict={x: inputX })

array([[  2.29202034e-22,   1.00000000e+00],
       [  8.42769978e-21,   1.00000000e+00],
       [  4.52256456e-37,   1.00000000e+00],
       [  2.61883761e-11,   1.00000000e+00],
       [  1.93388111e-26,   1.00000000e+00],
       [  4.40430760e-28,   1.00000000e+00],
       [  8.80797812e-25,   1.00000000e+00],
       [  3.19818366e-10,   1.00000000e+00],
       [  4.09291630e-19,   1.00000000e+00],
       [  5.14812415e-10,   1.00000000e+00],
       [  9.48859252e-25,   1.00000000e+00],
       [  4.32387823e-31,   1.00000000e+00],
       [  7.20989368e-34,   1.00000000e+00],
       [  2.12250827e-36,   1.00000000e+00],
       [  8.30280278e-21,   1.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00],
       [  5.24124247e-04,   9.99475896e-01],
       [  1.40016421e-23,   1.00000000e+00],
       [  3.82262387e-33,   1.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00],
       [  1.89974390e-24,   1.00000000e+00],
       [  2.06476629e-16,   1.00000000e+00],
       [  

So It's guessing they're all good houses. That makes it get 7/10 correct. Not terribly impressive. A model with a hidden layer should do better, I guess.

Btw, this is how I calculated the softmax values in the post:

In [89]:
sess.run(tf.nn.softmax([1., 2.]))

array([ 0.26894143,  0.7310586 ], dtype=float32)